# Indexación

## Introducción

En este notebook vamos a:
* Cargar los datos del CSV.
* Crear los `Documents` a partir de las filas del CSV.
* Dividir los `Documents` en chunks y generar los embeddings de esos chunks.
* Crear un `VectorStoreIndex` para que se pueda hacer la recuperación semántica.
* Persistir los embeddings y metadatos en Chroma.

Usaremos LlamaIndex como framework de indexación y Chroma como vectorstore.

## Documentos de referencia
* [Indexing | LlamaIndex Python Documentation](https://developers.llamaindex.ai/python/framework/module_guides/indexing/)
* [Storing | LlamaIndex Python Documentation](https://developers.llamaindex.ai/python/framework/understanding/rag/storing/)

## Imports
* Importamos `pandas` para trabajar con el CSV.
* De `llama_index.core` importamos:
    * [Document](https://developers.llamaindex.ai/python/framework/module_guides/loading/documents_and_nodes/): lo usamos para representar cada fila del CSV como un objeto estructurado.
    * [VectorStoreIndex](https://developers.llamaindex.ai/python/framework/module_guides/indexing/vector_store_index/): es la estructura que organiza los documentos y permite la recuperación semántica sobre un vectorstore.
    * [StorageContext](https://developers.llamaindex.ai/python/framework-api-reference/storage/storage_context/): permite configurar cómo y dónde se almacenan los datos del índice.
* De `llama_index.embeddings` nos traemos [SentenceTransformers de Hugging Face](https://huggingface.co/sentence-transformers) para convertir los chunks en vectores.
* De `llama_index.vector_stores` importamos [ChromaVectorStore](https://developers.llamaindex.ai/python/framework/integrations/vector_stores/chroma_metadata_filter/) para integrar LlamaIndex con Chroma.
* Importamos `chromadb` para inicializar y gestionar el cliente de Chroma, que actúa como base de datos vectorial persistente.

In [1]:
import pandas as pd
from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

In [2]:
# Configurar embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [3]:
# Crear lista documents
df = pd.read_csv("../data/charities.csv")

documents = []
for _, r in df.iterrows():
    text = f"""Name: {r['name']}
        Cause: {r['cause']}
        Regions: {r['regions']}
        Primary source: {r['source_primary']}

        Evidence:
        {r['evidence_text']}

        Transparency notes:
        {str(r.get('transparency_notes', ''))} # evitamos que falle si vacío
        """
    metadata = {
                "id": r["id"],
                "name": r["name"],
                "cause": r["cause"],
                "regions": r["regions"],
                "source_primary": r["source_primary"],
                "source_url": r["source_url"],
                "transparency_url": r.get("transparency_url", ""), # evitamos que falle si vacío
                }
    documents.append(Document(text=text, metadata=metadata))

In [4]:
# Guardar en disco
# https://developers.llamaindex.ai/python/framework/integrations/vector_stores/chromaindexdemo/#basic-example-including-saving-to-disk
chroma_client = chromadb.PersistentClient(path="../data/chroma_db")
chroma_collection = chroma_client.get_or_create_collection("charities")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, embed_model=embed_model,)

# Persiste
index.storage_context.persist(persist_dir="../data/index_store")